# Sharding
- Database partitioning known as "sharding" divides big databases into smaller, easier-to-manage segments aka. "shards."
- a strategy used to horizontally partition data across multiple databases or servers
- Purpose:
  - Data Volume Growth
  - Scalability
  - Fault Isolation

- Use this when:
  - working with large datasets that grow fast — time-series data, logs, users, events.
  - need horizontal scaling to distribute load or manage operational limits.
  - want to run queries in parallel without bottlenecking on a single chunk of data.
  - building on top of infrastructure like BigQuery, Hive, or Spark — where partitioning isn't optional, it's survival.

- Type:
  - Range-Based Sharding
```
SELECT * FROM users WHERE user_id BETWEEN 1 AND 1000;
```

  - Hash-Based Sharding
```
SELECT * FROM users WHERE MOD(user_id, 3) = 1;  -- For shard 1
SELECT * FROM users WHERE MOD(user_id, 3) = 2;  -- For shard 2
```
query fetches rows from the users table for which remainder of dividing user_id by 3 is 2 (e.g., user IDs 2, 5, 8). It retrieves the data from shard 2.


  - Directory-Based Sharding
    - aka.lookup-based sharding
    - map your shards with something called a lookup table.
    - to shard a table based on region so that all rows in a certain region end up on the same shard, you can set up a lookup table that maps that region to the specific shard.

  - MsSQL sharding implementation


```
// Example in Node.js to handle sharding logic
import mysql from 'mysql2/promise';

// Shard connections
const shards = [
  mysql.createConnection({ host: 'shard1.db.com', user: 'root', database: 'db1' }),
  mysql.createConnection({ host: 'shard2.db.com', user: 'root', database: 'db2' }),
];

// Function to get shard by user ID (Range-based sharding)
function getShardByUserId(userId: number) {
  if (userId <= 1000) return shards[0]; // Shard 1
  else return shards[1]; // Shard 2
}

// Query a user by ID
async function getUserById(userId: number) {
  const shard = getShardByUserId(userId);
  const [rows] = await shard.query('SELECT * FROM users WHERE user_id = ?', [userId]);
  return rows;
}
```

**Best Practices for Sharding in MySQL**

Choose an Effective Shard Key:
- shard key should ensure an even distribution of data across shards to avoid hotspots. Choose keys that are unlikely to create an imbalanced distribution (e.g., avoid timestamps as shard keys in highly active systems).

↳ Monitor and Adjust Shards:
- Continuously monitor your shards for performance issues. If a particular shard becomes too large, consider re-sharding or adjusting your shard key distribution.

↳ Automate Rebalancing:
- Implement mechanisms to rebalance data automatically when a shard becomes overloaded. Tools like Vitess can help manage rebalancing for MySQL-based systems.

↳ Backup and Recovery:
- Ensure each shard is backed up separately and that you have a recovery strategy in place in case of data loss on a specific shard.


# Partitioning vs Sharding
Partitioning is logical. Sharding is physical.

In [2]:
# Example directory-based sharding
# Example lookup table
lookup_table = {
    'East': 'Shard_1',
    'West': 'Shard_2',
    'North': 'Shard_3',
    'South': 'Shard_4'
}

# Shards represented as separate "databases" (simplified for demonstration)
shards = {
    'Shard_1': [],
    'Shard_2': [],
    'Shard_3': [],
    'Shard_4': []
}

def insert_member(name, court):
    # Determine the shard
    shard = lookup_table.get(court)
    if not shard:
        raise Exception(f"No shard found for court: {court}")

    # Insert into the corresponding shard
    shards[shard].append({'name': name, 'court': court})
    print(f"Inserted {name} into {shard} for court {court}")

def query_members_by_court(court):
  # Determine the shard
  shard = lookup_table.get(court)
  if not shard:
      raise Exception(f"No shard found for court: {court}")

  # Query the corresponding shard
  results = [member for member in shards[shard] if member['court'] == court]
  return results

# Insert data
insert_member('Alice', 'East')
insert_member('Bob', 'West')
insert_member('Charlie', 'North')
insert_member('Diana', 'South')

# Query data
print(query_members_by_court('East'))  # [{'name': 'Alice', 'court': 'East'}]

Inserted Alice into Shard_1 for court East
Inserted Bob into Shard_2 for court West
Inserted Charlie into Shard_3 for court North
Inserted Diana into Shard_4 for court South
[{'name': 'Alice', 'court': 'East'}]


In [6]:
# Hash-based shardig
import hashlib

# Example shards
shards = {
    0: [],
    1: [],
    2: [],
    3: []
}

# Function to determine shard using MD5 and modulo
def get_shard(id, num_shards=4):
    # Hash the id using MD5
    hash_object = hashlib.md5(str(id).encode())
    hash_value = hash_object.hexdigest()

    # Convert the first 8 characters of the hash to decimal
    decimal_value = int(hash_value[:8], 16)

    print(hash_object, hash_value, decimal_value)

    # Compute shard by modding with the number of shards
    shard_number = decimal_value % num_shards
    return shard_number

# Insert into the appropriate shard
def insert_member(id, name):
    shard_number = get_shard(id)
    shards[shard_number].append({'id': id, 'name': name})
    print(f"Inserted {name} (id={id}) into Shard {shard_number}")

# Example inserts
insert_member(12345, "Alice")  # Goes to Shard 2
insert_member(67890, "Bob")    # Goes to Shard 3

# Output the shards
print(shards)

<md5 _hashlib.HASH object @ 0x7efbd0bf29b0> 827ccb0eea8a706c4c34a16891f84e7b 2189216526
Inserted Alice (id=12345) into Shard 2
<md5 _hashlib.HASH object @ 0x7efbd0bf2590> 1e01ba3e07ac48cbdab2d3284d1dd0fa 503429694
Inserted Bob (id=67890) into Shard 2
{0: [], 1: [], 2: [{'id': 12345, 'name': 'Alice'}, {'id': 67890, 'name': 'Bob'}], 3: []}
